<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/%EB%B6%80%EC%A1%B1%ED%95%9C%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module

In [5]:
import tensorflow as tf
import numpy as np
import random

# Dataset

In [2]:
cifar10 = tf.keras.datasets.cifar10

In [3]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [4]:
X_train, X_test = X_train/255., X_test/255.

## Datset 부족하게 만들기

In [6]:
x_train_small = list()
y_train_small = list()

for x,y in zip(X_train,y_train):
  # 0번에 대해서는 10%만 사용, 1번은 모두 사용
  if (y == 0 and random.randint(0,100) < 10) or y == 1:
    x_train_small.append(x[:])
    y_train_small.append(y)

In [7]:
x_test_small = list()
y_test_small = list()

for x,y in zip(X_test,y_test):
  if y == 0 or y== 1:
    x_test_small.append(x[:])
    y_test_small.append(y)

In [12]:
X_train = np.stack(x_train_small,axis=0)
y_train = np.stack(y_train_small,axis=0)

In [13]:
X_test = np.stack(x_test_small,axis=0)
y_test = np.stack(y_test_small,axis=0)

In [14]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(1000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1000).batch(32).prefetch(1024)

# 모형

In [15]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(1024,activation='relu')
    self.dense2 = tf.keras.layers.Dense(1,activation='sigmoid')

  def call(self,x,training=False,mask=None):
    x = self.flatten(x)
    x = self.dense1(x)
    out = self.dense2(x)
    return out

# 학습

In [16]:
model = MyModel()

In [17]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])

In [18]:
model.fit(train_ds,validation_data=test_ds,epochs=10)

Epoch 1/10
172/172 [==============================] - 9s 43ms/step - loss: 0.3621 - accuracy: 0.9077 - precision: 0.9181 - recall: 0.9866 - val_loss: 0.6198 - val_accuracy: 0.6980 - val_precision: 0.6284 - val_recall: 0.9690
Epoch 2/10
172/172 [==============================] - 7s 41ms/step - loss: 0.2510 - accuracy: 0.9123 - precision: 0.9283 - recall: 0.9792 - val_loss: 0.8131 - val_accuracy: 0.6185 - val_precision: 0.5682 - val_recall: 0.9870
Epoch 3/10
172/172 [==============================] - 8s 45ms/step - loss: 0.2273 - accuracy: 0.9203 - precision: 0.9333 - recall: 0.9826 - val_loss: 0.8899 - val_accuracy: 0.6240 - val_precision: 0.5717 - val_recall: 0.9890
Epoch 4/10
172/172 [==============================] - 10s 60ms/step - loss: 0.2062 - accuracy: 0.9279 - precision: 0.9405 - recall: 0.9830 - val_loss: 1.1474 - val_accuracy: 0.5600 - val_precision: 0.5321 - val_recall: 0.9940
Epoch 5/10
172/172 [==============================] - 8s 47ms/step - loss: 0.2099 - accuracy: 0.927

정밀도가 낮고 재현율이 낮음

0번이 적기 때문에 다 1로 찍음

# Smote

In [20]:
!pip install imblearn

In [22]:
from imblearn.over_sampling import BorderlineSMOTE

In [23]:
smote = BorderlineSMOTE()

In [26]:
x_train = X_train.reshape((X_train.shape[0],X_train.shape[1] * X_train.shape[2] * X_train.shape[3]))
x_test = X_test.reshape((X_test.shape[0],-1))

In [28]:
x_train,y_train = smote.fit_resample(x_train,y_train)

In [29]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32).prefetch(1024)

In [30]:
model.fit(train_ds,validation_data=test_ds,epochs=10)

Epoch 1/10
313/313 [==============================] - 17s 50ms/step - loss: 0.1528 - accuracy: 0.9468 - precision: 0.9436 - recall: 0.9502 - val_loss: 0.7434 - val_accuracy: 0.7305 - val_precision: 0.9390 - val_recall: 0.4930
Epoch 2/10
313/313 [==============================] - 17s 53ms/step - loss: 0.1609 - accuracy: 0.9424 - precision: 0.9397 - recall: 0.9452 - val_loss: 1.0636 - val_accuracy: 0.6725 - val_precision: 0.9551 - val_recall: 0.3620
Epoch 3/10
313/313 [==============================] - 14s 45ms/step - loss: 0.1985 - accuracy: 0.9389 - precision: 0.9386 - recall: 0.9390 - val_loss: 0.7584 - val_accuracy: 0.7295 - val_precision: 0.9355 - val_recall: 0.4930
Epoch 4/10
313/313 [==============================] - 14s 44ms/step - loss: 0.1448 - accuracy: 0.9490 - precision: 0.9488 - recall: 0.9490 - val_loss: 0.9877 - val_accuracy: 0.6950 - val_precision: 0.9514 - val_recall: 0.4110
Epoch 5/10
313/313 [==============================] - 15s 48ms/step - loss: 0.1638 - accuracy: 0

정밀도와 검증 정확도가 올라감